Prepare CSV file (employees.csv):

id,name,age,department,salary,join_date
1,John,34,IT,75000,2015-06-01
2,Sara,28,HR,58000,2019-09-15
3,Michael,45,Finance,120000,2010-01-10
4,Karen,29,IT,70000,2020-02-19
5,David,38,Finance,90000,2017-08-23
6,Linda,33,HR,60000,2018-12-05
7,James,41,IT,110000,2013-04-15
8,Emily,27,HR,52000,2021-06-20
9,Robert,36,Finance,105000,2016-11-30

Read above csv file as pyspark dataframe and solve below problems.


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

# AFter creating csv file and loading it into databricks
file_location = "/FileStore/tables/employees.csv"

df = spark.read.format("csv").option("header", True).option("inferSchema", True).load(file_location)

display(df)

id,name,age,department,salary,join_date
1,John,34,IT,75000,2015-06-01
2,Sara,28,HR,58000,2019-09-15
3,Michael,45,Finance,120000,2010-01-10
4,Karen,29,IT,70000,2020-02-19
5,David,38,Finance,90000,2017-08-23
6,Linda,33,HR,60000,2018-12-05
7,James,41,IT,110000,2013-04-15
8,Emily,27,HR,52000,2021-06-20
9,Robert,36,Finance,105000,2016-11-30



1. Display records of employees aged above 30.
Use the DataFrame filter transformation to extract employees where age > 30.

In [0]:
df.filter(col('age')>30).display()

id,name,age,department,salary,join_date
1,John,34,IT,75000,2015-06-01
3,Michael,45,Finance,120000,2010-01-10
5,David,38,Finance,90000,2017-08-23
6,Linda,33,HR,60000,2018-12-05
7,James,41,IT,110000,2013-04-15
9,Robert,36,Finance,105000,2016-11-30



2. Find the average salary of employees in each department.
Use the groupBy transformation on the department column and calculate the average salary.

In [0]:
df.groupBy("Department").agg(avg("salary").alias("Average_Salary")).display()

Department,Average_Salary
HR,56666.666666666664
Finance,105000.0
IT,85000.0



3. Add a column experience indicating the number of years an employee has been working.
Derive this using the join_date column and the current date.

In [0]:
df = df.withColumn("experience",(datediff(current_date(), col("join_date")) / 365).cast("int")).display()

id,name,age,department,salary,join_date,experience
1,John,34,IT,75000,2015-06-01,9
2,Sara,28,HR,58000,2019-09-15,5
3,Michael,45,Finance,120000,2010-01-10,14
4,Karen,29,IT,70000,2020-02-19,4
5,David,38,Finance,90000,2017-08-23,7
6,Linda,33,HR,60000,2018-12-05,6
7,James,41,IT,110000,2013-04-15,11
8,Emily,27,HR,52000,2021-06-20,3
9,Robert,36,Finance,105000,2016-11-30,8



4. Find the top 3 highest-paid employees.
Sort the DataFrame in descending order by salary and use the limit action.


In [0]:
df.orderBy(col("salary").desc()).limit(3).display()

id,name,age,department,salary,join_date
3,Michael,45,Finance,120000,2010-01-10
7,James,41,IT,110000,2013-04-15
9,Robert,36,Finance,105000,2016-11-30



5. Identify the department with the highest total salary.
Aggregate the salary data by department and find the department with the maximum total.

In [0]:
df.groupBy("department").sum("salary").orderBy(col("sum(salary)").desc()).limit(1).display()

department,sum(salary)
Finance,315000



6. Create a new DataFrame with employees earning more than the average salary.
First, calculate the average salary, then filter employees earning above that average.

In [0]:
avg_salary = df.select(avg("salary")).first()[0]
df.filter(col("salary") > avg_salary).display()

id,name,age,department,salary,join_date
3,Michael,45,Finance,120000,2010-01-10
5,David,38,Finance,90000,2017-08-23
7,James,41,IT,110000,2013-04-15
9,Robert,36,Finance,105000,2016-11-30



7. Rename the column name to employee_name.
Use the withColumnRenamed transformation.

In [0]:
df.withColumnRenamed("name", "employee_name").display()

id,employee_name,age,department,salary,join_date
1,John,34,IT,75000,2015-06-01
2,Sara,28,HR,58000,2019-09-15
3,Michael,45,Finance,120000,2010-01-10
4,Karen,29,IT,70000,2020-02-19
5,David,38,Finance,90000,2017-08-23
6,Linda,33,HR,60000,2018-12-05
7,James,41,IT,110000,2013-04-15
8,Emily,27,HR,52000,2021-06-20
9,Robert,36,Finance,105000,2016-11-30



8. Find the number of employees in each department.
Use groupBy and count transformations.

In [0]:
df.groupBy("department").agg(count("*").alias("employee_count")).display()

department,employee_count
HR,3
Finance,3
IT,3



9. Select and display only the id and name columns.
Use the select transformation to project only specific columns.

In [0]:
df.select("name","age").display()

name,age
John,34
Sara,28
Michael,45
Karen,29
David,38
Linda,33
James,41
Emily,27
Robert,36



10. Check for any null values in the dataset.
Use the isNull method to check for nulls across columns.

In [0]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).display()

id,name,age,department,salary,join_date
0,0,0,0,0,0
